In [ ]:
# Adjustable Privacy - train_on_original.ipynb
# - Train a machine (weak adversary) on original (image) dataset to infer a specific feature.
# - Uses image dataset (CelebA).
# - Saves models after each epoch number (to google drive and locally).
# - It can stop and resume training.
# - Draws loss and accuracy plots and saves them (to google drive).
# - Also it can load models and draw plots (from google drive).
# - You can manage notebook parameters in parser block

In [ ]:
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.utils.data import random_split
from torchvision import datasets, transforms, models
import torchvision.utils as vutils
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import itertools
import random
import shutil
from zipfile import ZipFile
import os
from tqdm import tqdm
from collections import OrderedDict
import time
from math import floor
import argparse

In [ ]:
# Parser
parser = argparse.ArgumentParser(description='Adjustable Privacy - Train a machine on original (image) dataset to infer a specific feature. '
                                 + 'Uses image dataset (CelebA). '
                                 + 'Saves models after each epoch number (to google drive and locally). '
                                 + 'It can stop and resume training.'
                                 + 'Draws loss and accuracy plots and saves them (to google drive and locally). '
                                 + 'Also it can load models and draw plots (from google drive).')

parser.add_argument('--resume', default = False, help = 'Accepts "True" or "False". ')
parser.add_argument('--last_epoch', type=int, default = 0, help = 'In case of resumming training use last saved epoch number and in case of loading a model, set to model number.')
parser.add_argument('--target_index', type=int, required=True, help = 'gender(20), smiling(31), open mouth(21), and high chickbone(19)')
parser.add_argument('--save_path', type=str, required=True, help = 'Full path on your google drive to save model and plots. And also load from it. Like "drive/MyDrive/adjustable-privacy/Models/CelebA-G/"')
parser.add_argument('--epoch_numbers', type=int, default = 50, help = 'Number of epochs to train model. (when you want load a model, it should set to that model number)')
parser.add_argument('--download_dataset', default = False, help = 'Accepts "True" or "False". Download CelebA dataset or use CelebA shared directory.')
parser.add_argument('--dataset_path', type=str, default = "", help = '(if --download_dataset=False) Full path on your google drive to CelebA shared directory shortcut. Like "drive/MyDrive/adjustable-privacy/Datasets/CelebA/"')

command_string = "--resume False" \
" --last_epoch 0" \
" --target_index 20" \
" --save_path drive/MyDrive/adjustable-privacy/Models/CelebA-G/" \
" --epoch_numbers 50"\ 
" --download_dataset False" \
" --dataset_path drive/MyDrive/adjustable-privacy/Datasets/CelebA/"

args = parser.parse_args(command_string.split())

In [ ]:
# Hyper parameters:
isFirstRun = args.resume=='False'
lastRunEpochNumber = args.last_epoch
manual_seed = 20
image_size = 64
use_whole_dataset = True
usage_percent = 1.0
learning_rate = 0.001 #0.2
batch_size = 64
celeba_male_index = 20
celeba_smiling_index = 31
celeba_mouth_open_index = 21
celeba_high_cheekbone_index = 19
using_index = args.target_index
saving_path = args.save_path
# Number of workers for dataloader
workers = 2
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available.
ngpu = 1
# Size of feature maps in encoder
nef = 64
# Number of channels in the training images. For color images this is 3
nc = 3
# Number of training epochs
num_epochs = args.epoch_numbers
data_dir = 'celeba'
download_dataset = args.download_dataset=='True'
dataset_folder_path = args.dataset_path

In [ ]:
# Check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# getting dataset ready from shared directory
if not download_dataset:
    dataset_zip_path = dataset_folder_path + '/Img/img_align_celeba.zip'
    list_eval_partition_path = dataset_folder_path + '/Eval/list_eval_partition.txt'
    identity_celeba_path = dataset_folder_path + '/Anno/identity_CelebA.txt'
    list_attr_celeba_path = dataset_folder_path + '/Anno/list_attr_celeba.txt'
    list_bbox_celeba_path = dataset_folder_path + '/Anno/list_bbox_celeba.txt'
    list_landmarks_align_celeba_path = dataset_folder_path + '/Anno/list_landmarks_align_celeba.txt'

    try:
      os.mkdir(data_dir)
      print("data folder created successfully")
    except OSError as e:
      print("Error: %s" % (e.strerror))

    shutil.copyfile(dataset_zip_path, data_dir + r'/img_align_celeba.zip')
    shutil.copyfile(list_eval_partition_path, data_dir + r'/list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, data_dir + r'/identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, data_dir + r'/list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, data_dir + r'/list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, data_dir + r'/list_landmarks_align_celeba.txt')

    try:
        shutil.rmtree(data_dir + r'/img_align_celeba')
        print("old unzipped directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

    archive = data_dir + r'/img_align_celeba.zip'
    with ZipFile(archive, 'r') as zip:
       zip.extractall(data_dir)

In [ ]:
# make saving path dir
try:
    os.makedirs(saving_path)
    print("saving_path directory created successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

In [ ]:
# Define transforms
train_transforms = transforms.Compose([transforms.Resize(image_size),
                                       transforms.CenterCrop(image_size),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
# Function - use some percent of data
def shorten_dataset(dataset, usage_percent=1.0):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed)) 
  return used_dataset

In [ ]:
# Load Datas
train_set = datasets.CelebA(root='', download=download_dataset, split='train', target_type=["attr", "identity"], transform=train_transforms)
test_set = datasets.CelebA(root='', download=download_dataset, split='test', target_type=["attr", "identity"], transform=test_transforms)
valid_set = datasets.CelebA(root='', download=download_dataset, split='valid', target_type=["attr", "identity"], transform=test_transforms)

# shorten Dataset
if not use_whole_dataset:
  train_set = shorten_dataset(train_set, usage_percent)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=workers, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=workers, drop_last=True)

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

In [ ]:
# custom weights initialization
def weights_init(m):
    classname = m.__class__.__name__
    print(classname)
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Model 
class MyModel(nn.Module):
    def __init__(self, ngpu):
        super(MyModel, self).__init__()
        self.ngpu = ngpu
        
        # input is nc x 64 x 64 
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024
        self.fllc_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features1 = nn.Dropout(p=0.5)
        self.fllc_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features2 = nn.Dropout(p=0.5)
        self.fllc_features3 = nn.Linear(nef*4, nef)
        self.actv_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_features4 = nn.Linear(nef, 2)
        self.actv_features4 = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        y1 = self.fllc_features1(x)
        y1 = self.actv_features1(y1)
        y1 = self.dropout_features1(y1)
        y1 = self.fllc_features2(y1)
        y1 = self.actv_features2(y1)
        y1 = self.dropout_features2(y1)
        y1 = self.fllc_features3(y1)
        y1 = self.actv_features3(y1)
        y1 = self.fllc_features4(y1)
        y1 = self.actv_features4(y1)
        return y1

In [ ]:
# Create the model instance
modelInstance = MyModel(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    modelInstance = nn.DataParallel(modelInstance, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
modelInstance.apply(weights_init)

In [ ]:
# total parameters
total_params = sum(p.numel() for p in modelInstance.parameters())
print(f"{total_params:,} total parameters.")

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(modelInstance.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [ ]:
# Function - Save:
def save_model(name, number, model, res):
  checkpoint = {'res': res, 'state_dict': model.state_dict()}
  torch.save(checkpoint, saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth')
  return True

In [ ]:
# Function - Load:
def load_model(name, number, model, device):
  checkpoint = torch.load(saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,'res':res}

In [ ]:
# Save Start Checkpoint
if(isFirstRun):
  ins_res = {'train_losses': [],
             'valid_losses': [],
             'test_y1_acc': [],
             'epoch_number': 0,
           };
  save_model('ins', 0, modelInstance, ins_res)

In [ ]:
# Load Last Checkpoint:
ins_load = load_model('ins', lastRunEpochNumber, modelInstance, device)
train_losses = ins_load['res']['train_losses']
valid_losses = ins_load['res']['valid_losses']
test_y1_acc = ins_load['res']['test_y1_acc']
last_epoch = ins_load['res']['epoch_number']

In [ ]:
# Function - training function
def fit(model, train_loader, optimizer, criterion):
    print('Training')
    model.train()
    train_loss = 0.0
    prog_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in prog_bar:
        inputs, labels = data[0], data[1]
        inputs = inputs.to(device)
        main_target = labels[0][:, using_index]
        main_target = main_target.to(device)
        model.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, main_target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()          
    train_loss = train_loss / len(train_loader)
    return train_loss

In [ ]:
# Function - validation function
def validate(model, valid_loader, criterion):
    print('\nValidating')
    model.eval()
    valid_loss = 0.0
    prog_bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            main_target = labels[0][:, using_index]
            main_target = main_target.to(device)
            outputs = model.forward(inputs)
            loss = criterion(outputs, main_target)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(valid_loader)
        return valid_loss

In [ ]:
# Calc Accuracy
def calcAccuracyTest(model, test_loader):
    model.to(device)
    print("\nCalculating Accuracy...")
    model.eval()
    y1_accuracy = 0
    prog_bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            main_target = labels[0][:, using_index]
            main_target = main_target.to(device)
            output = model(inputs)
            ps_y1 = torch.exp(output)
            top_p_y1, top_class_y1 = ps_y1.topk(1, dim=1)
            equals_y1 = top_class_y1 == main_target.view(*top_class_y1.shape)
            acc_y1 = equals_y1.sum().item()
            y1_accuracy += (acc_y1 / len(equals_y1))            
    y1_accuracy = y1_accuracy / len(test_loader)
    return y1_accuracy

In [ ]:
# Training Loop
modelInstance.to(device)
save_every_epoch = 1
start = time.time()
print("Starting Training Loop...")
for epoch in range(last_epoch+1, num_epochs+1):
    print(f"Epoch {epoch}/{num_epochs}: ")
    train_loss = fit(modelInstance, train_loader, optimizer, criterion)
    valid_loss = validate(modelInstance, valid_loader, criterion)
    y1_accuracy = calcAccuracyTest(modelInstance, test_loader)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    test_y1_acc.append(y1_accuracy)
    ins_res = {'train_losses': train_losses,
               'valid_losses': valid_losses,
               'test_y1_acc': test_y1_acc,
               'epoch_number': epoch
                }
    if epoch % save_every_epoch == 0:
        save_model('ins', epoch, modelInstance, ins_res)
        
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Valid Loss: {valid_loss:.4f}")
    print(f"Accuracy on Testset: {y1_accuracy:.4f}")
end = time.time()
print(f"Training time: {(end-start)/60:.3f} minutes")
print('TRAINING COMPLETE')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print('Loss plot...')
# loss plots
plt.figure(figsize=(10,7))
plt.title("Train-Valid Loss Trend")
plt.plot(train_losses, color='green', label='Training Loss')
plt.plot(valid_losses, color='blue', label='Validation Loss')
plt.legend(frameon=False)
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.savefig(saving_path + "loss_plot.png")
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Accuracy Trend")
plt.plot(test_y1_acc, color='green', label='Testset Accuracy')
plt.legend(frameon=False)
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.savefig(saving_path + "accuracy_test_plot.png")
plt.show()